In [1]:
import pandas as pd 
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.runnables import chain
from bs4 import BeautifulSoup
import requests
from pydantic import BaseModel, Field

In [2]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests


In [3]:
url_dataset = '../../datasets/relevantes/dataset_links.csv'

dataset = pd.read_csv(url_dataset)

dataset.head()

,link
0,https://bit.ly/3lUcz1t
1,https://bit.ly/3g4FlsE
2,https://bit.ly/3g7mn4w
3,https://bit.ly/3sjQ0UY
4,https://bit.ly/3m6uVwh


In [4]:
dataset['link'][0]

'https://bit.ly/3lUcz1t'

# na mão

In [4]:
def fetch_page_content(url):
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() 
        soup = BeautifulSoup(response.content, "html.parser")
        text = soup.get_text(separator="\n").strip()
        return text if text else "Conteúdo vazio ou indisponível."
    except Exception as e:
        return f"Erro ao acessar o link: {e}"

In [19]:
class Response(BaseModel):
    summary: str = Field(description="Resumo da notícia fornecida.", required=True)

In [20]:
summary_prompt = PromptTemplate(
    input_variables=["content"],
    template="""
Você é um assistente especializado em gerar resumos detalhados.

Seu objetivo é fornecer um resumo claro e completo, destacando os principais pontos, argumentos e informações relevantes do conteúdo. O resumo deve ser informativo e bem estruturado, com foco nas ideias principais. 

Lembre-se de que o resumo deve ser coeso e coerente, respeitando um limite máximo de 350 palavras, para permitir um pouco mais de profundidade e detalhes.

Texto:
{content}
"""
)

llm = ChatOllama(model="llama3.1", format="json", temperature=0.1)

structured_llm = llm.with_structured_output(Response)

summary_chain = summary_prompt | structured_llm


In [5]:
def summarize_web_content(url):
    page_content = fetch_page_content(url)
    
    if "Erro" in page_content or "Conteúdo vazio" in page_content:
        return page_content  
    
    summary = summary_chain.invoke(page_content)
    return summary


In [6]:
url = dataset['link'][0]
response = summarize_web_content(url)

In [7]:
response

'Erro ao acessar o link: 403 Client Error: Forbidden for url: https://www.campograndenews.com.br/cidades/capital/queimada-atinge-casa-de-festas-e-empresario-estima-r-80-mil-de-prejuizo'

# usando biblioteca  

In [27]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = dataset['link'][0]

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

In [28]:
main_content = doc.page_content.strip()

print(f"{doc.metadata}\n")
print(main_content[:500])

# with open("document.txt", "w", encoding="utf-8") as file:
#     file.write(main_content)


{'source': 'https://bit.ly/3lUcz1t', 'title': 'Queimada atinge casa de festas e empresário estima R$ 80 mil de prejuízo - Capital - Campo Grande News', 'description': 'Rotineiro durante época de seca, incêndio em vegetação se espalhou e destruiu uma casa de festas nas Moreninhas durante a tarde desta terça-feira (10). Ap...', 'language': 'pt-br'}

Queimada atinge casa de festas e empresário estima R$ 80 mil de prejuízo - Capital - Campo Grande News  
   


menu
Menu








ACOMPANHE-NOS
   



 (67) 99669-9563
 DEZEMBRO, QUARTA  18
   CAMPO GRANDE 28º

searchBusca



As mais Mais lidas

Motociclista de aplicativo morre ao ser atingido por moto de alta cilindradaEmpresário "acorrenta" carro que parou em vaga exclusiva para clientesAcidente mata 2 adultos, 1 criança e deixa 3 feridosAcidente que matou motociclista de aplicativo foi gravado


In [9]:
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(class_="theme-doc-markdown markdown"),
    },
    bs_get_text_kwargs={"separator": " | ", "strip": True},
)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

In [10]:
print(f"{doc.metadata}\n")
print(doc.page_content[:500])

{'source': 'https://bit.ly/3lUcz1t'}




In [4]:
from langchain_unstructured import UnstructuredLoader

page_url = "https://siscad.ufms.br/"

loader = UnstructuredLoader(web_url=page_url)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

In [5]:
for doc in docs[:5]:
    print(doc.page_content)

Bem-vindo ao Portal de Graduação!
Para acessar o sistema acadêmico, clique no botão abaixo:
Acessar o Siscad
Para outros você pode conferir abaixo:
Bem-vindo ao Portal de Graduação!


In [6]:
for doc in docs[:5]:
    print(f'{doc.metadata["category"]}: {doc.page_content}')

Title: Bem-vindo ao Portal de Graduação!
Title: Para acessar o sistema acadêmico, clique no botão abaixo:
Title: Acessar o Siscad
NarrativeText: Para outros você pode conferir abaixo:
Title: Bem-vindo ao Portal de Graduação!


 # Refazendo scraping

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--disable-javascript")
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()

url = dataset['link'][0]

driver.get(url)

print(driver.page_source)

driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.116)
Stacktrace:
#0 0x58f2e88ae6da <unknown>
#1 0x58f2e83c2f80 <unknown>
#2 0x58f2e83992ef <unknown>
#3 0x58f2e843beed <unknown>
#4 0x58f2e8451f49 <unknown>
#5 0x58f2e8433953 <unknown>
#6 0x58f2e840272e <unknown>
#7 0x58f2e840379e <unknown>
#8 0x58f2e8879f1b <unknown>
#9 0x58f2e887deb8 <unknown>
#10 0x58f2e886742c <unknown>
#11 0x58f2e887ea37 <unknown>
#12 0x58f2e884bfef <unknown>
#13 0x58f2e889cad8 <unknown>
#14 0x58f2e889cca0 <unknown>
#15 0x58f2e88ad556 <unknown>
#16 0x7f1380c94ac3 <unknown>
